## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認し、ファイルロードからクリーニングまでのプロセスを決定します。

<details>
<summary><b>結果</b></summary>


</details>


### 初期設定

In [9]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.data_cleaning_utils as cu
import src.eda_utils as eu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


In [10]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
#display(settings)

### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめ、プロセスを確定します。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。  
修正処理は不要です。
</details>

In [11]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1,force_str=True)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\n次に欠けてるファイルがないか確認します")
results = [dl.chk_file_missing(df) for df in df_DATAs_BY_ALL_FILEs.values()]

print("\n重複ファイルがないか確認します")
dl.chk_duplicate_files_df(dl.chk_duplicate_dfs(df_DATAs_BY_ALL_FILEs, False, False))

print("\nデータロードの確認を終了します。")



ファイル毎にデータをロードします

次に欠けてるファイルがないか確認します

重複ファイルがないか確認します

データロードの確認を終了します。


### 欠損値表現の確認
欠損値の表現を確認し、修正プロセスを決定します。

<details>
<summary><b>結果</b></summary>

"-"が欠損値として使われています。  
これはsetting.yamlのnan_valuesに登録し欠損値として読み込みます。  
  
また、ゼロも使われています。  
ゼロは値として残し、必要に応じてfeatures_generatorで再計算し更新します。

以下がゼロが使われている項目です。  
{'fy-balance-sheet.csv': ['BPS', '短期借入金', '総資産', '自己資本比率', '長期借入金']  
'fy-cash-flow-statement.csv': ['営業CF', '営業CFマージン', '投資CF', '現金同等物', '設備投資', '財務CF']  
'fy-profit-and-loss.csv': ['EPS', 'ROA', 'ROE', '営業利益', '売上高', '純利益', '経常利益'],  
'fy-stock-dividend.csv': ['一株配当', '剰余金の配当', '純資産配当率', '総還元性向', '自社株買い', '配当性向']}
</details>

In [12]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1,force_str=True)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\nファイルごとに各文字列の個数をカウントします")
df_placeholder_counts = pd.DataFrame()
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = cu.chk_missing_values_expression(df, filename, year)
    df_placeholder_counts = pd.concat([df_placeholder_counts,df_tmp], axis=0)
#display(df_placeholder_counts)

print("\n欠損値と疑われる文字と該当する列を出力します。")
result = cu.chk_missing_and_suspect(df_placeholder_counts)
# 例: "欠損コードA" に該当する列リスト
display(result)


ファイル毎にデータをロードします

ファイルごとに各文字列の個数をカウントします


AttributeError: module 'src.data_cleaning_utils' has no attribute 'chk_missing_values_expression'

In [ ]:
# ゼロ値のチェック
#del list
merged = defaultdict(list)
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    merged[filename].append(df)
# ファイルごとに結合（行方向）
df_merged_by_file = {fname: pd.concat(dfs, ignore_index=True) for fname, dfs in merged.items()}
#display(df_final["fy-balance-sheet.csv"])

print("\nファイルごとにゼロを含むコードをチェックします。")
df_result = eu.chk_zero_data(df_merged_by_file)

df = df_result[df_result["ファイル名"] == settings["files"]["raw"][0]]
df = df[df["対象列"] == "総資産"].dropna(how="all",axis=1)
#display(df)

In [ ]:
# アルファベット値のチェック
print("")
print("アルファベットが含まれているコードを出力します。")
code_list = [
    code
    for (_, _), df in df_DATAs_BY_ALL_FILEs.items()
    for code in df["コード"].astype(str)[
        df["コード"].astype(str).str.contains("[A-Za-z]", na=False)].unique()
]
print(code_list)

### データ型の整合性確認
ロードしたファイルの方が各列で共通になるようにします。
<details>
<summary><b>結果</b></summary>
以下のように処理しています。

- コード：string
- 年度：timestump
- その他：float
</details>

In [13]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename, header=1, na_values=settings["na_values"])
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}


ファイル毎にデータをロードします


In [14]:
print("\n読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ")
for keys,df in df_DATAs_BY_ALL_FILEs.items():
    object_cols = df.select_dtypes(include='object').columns
    df_DATAs_BY_ALL_FILEs[keys].loc[:, object_cols] = df.loc[:, object_cols].replace("-", np.nan)



読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ


In [15]:
print("\nコードと年度以外をfloat化")
except_codes= ["コード", "年度"]
df_type_change = {
    key: cu.convert_columns_type(df, df.drop(columns=except_codes, errors="ignore").columns, "float", False)
    for key, df in df_DATAs_BY_ALL_FILEs.items()
}


コードと年度以外をfloat化


In [16]:
print("\nコードをstring化")
df_type_change = {
    key: df.assign(コード=df["コード"].astype("string"))
    for key, df in df_type_change.items()
}


コードをstring化


In [17]:
print("\n年度をdatetime化")
df_type_change = {
    key: df.assign(年度=pd.to_datetime(df["年度"], format="%Y/%m"))
    for key, df in df_type_change.items()
}


年度をdatetime化


In [18]:
print("\ndtypeの最終チェック")
for _, df in df_type_change.items():
    cu.chk_finale_dtype(df, settings["dtype_chk"]["raw"])


dtypeの最終チェック


### 初期品質の確認

指標となる以下の数を確認し、必要な処理のプロセスを決定します。

- コード
- 年度
- 列

<summary><b>結果</b></summary>

</details>

In [19]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

print("\n処理済みデータをロードします。")
df_PROCESSED = df_type_change
#display(df_PROCESSED)


print("\n企業コード数、年度の数、列数の年推移をグラフ化します。")
df_file_info = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED.items()
])
for col in ["code_counts", "year_counts", "column_counts"]:
    fig = px.line(df_file_info, x="year", y=col, color="file", title=col)
    #fig.show()

print("\n2025年に登録されている年度を表示します。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
display(df["年度"].unique())

print("\n最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"]["コード"])

print("\nコードを限定し、過去のデータファイルと同じかどうか調べます。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
display(df[df["コード"]=="130A"])
df = df_PROCESSED[(settings["files"]["raw"][0],2024)]
display(df[df["コード"]=="130A"])


処理済みデータをロードします。

企業コード数、年度の数、列数の年推移をグラフ化します。

2025年に登録されている年度を表示します。


<DatetimeArray>
['2025-03-01 00:00:00', '2024-12-01 00:00:00', '2025-02-01 00:00:00',
 '2025-05-01 00:00:00', '2025-06-01 00:00:00', '2024-10-01 00:00:00',
 '2024-11-01 00:00:00', '2023-12-01 00:00:00', '2024-08-01 00:00:00',
 '2025-01-01 00:00:00', '2025-04-01 00:00:00', '2024-09-01 00:00:00',
 '2024-07-01 00:00:00', '2024-03-01 00:00:00', '2023-03-01 00:00:00',
 '2023-06-01 00:00:00', '2023-09-01 00:00:00', '2025-07-01 00:00:00',
 '2024-05-01 00:00:00', '2024-02-01 00:00:00', '2023-02-01 00:00:00',
 '2025-08-01 00:00:00', '2024-01-01 00:00:00', '2023-08-01 00:00:00',
 '2023-11-01 00:00:00', '2024-04-01 00:00:00', '2024-06-01 00:00:00',
 '2023-05-01 00:00:00', '2023-04-01 00:00:00', '2023-07-01 00:00:00',
 '2023-01-01 00:00:00']
Length: 31, dtype: datetime64[ns]


最新年に登録されている過去の年度と過去のデータが同じか確認します
まずは、年度を限定して、コードの一覧を見ます

コードを限定し、過去のデータファイルと同じかどうか調べます。


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
1,130A,2024-12-01,2248958000.0,2209548000.0,2209548000.0,-203393000.0,<NA>,<NA>,340.61,98.2


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
1,130A,2024-12-01,2248000000.0,2209000000.0,2209548000.0,-203393000.0,<NA>,<NA>,340.61,98.2


In [20]:
print("\n重複した値を検出し、最新年度ファイルのものに更新します。")
df_result_lower, df_result_upper = cu.chk_duplicated_data(df_PROCESSED)
display(df_result_lower)

df_PROCESSED_UPDATED = cu.update_duplicated_data(df_PROCESSED, df_result_lower)


重複した値を検出し、最新年度ファイルのものに更新します。


,コード,年度,ファイル名,フォルダ名
0,130A,2024-12-01,fy-balance-sheet.csv,2025
1,1383,2024-10-01,fy-balance-sheet.csv,2025
2,138A,2024-11-01,fy-balance-sheet.csv,2025
3,1400,2023-12-01,fy-balance-sheet.csv,2025
4,1407,2024-08-01,fy-balance-sheet.csv,2025
...,...,...,...,...
4619,9972,2024-11-01,fy-stock-dividend.csv,2025
4620,9973,2023-12-01,fy-stock-dividend.csv,2025
4621,9977,2024-02-01,fy-stock-dividend.csv,2025
4622,9979,2024-08-01,fy-stock-dividend.csv,2025


In [21]:
print("\n重複したデータの更新元を消去します。")
files = df_result_lower["ファイル名"].unique()
years = df_result_lower["フォルダ名"].unique()
for filename in files:
    for year in years:
        df_update = df_result_lower.query("ファイル名 == @filename and フォルダ名 == @year")
        df_update_index = df_update.set_index(["コード", "年度"]).index
        df_target = df_PROCESSED_UPDATED[(filename, year)].set_index(["コード", "年度"])
        df_PROCESSED_UPDATED[(filename, year)] = df_target.drop(df_update_index).reset_index()


重複したデータの更新元を消去します。


In [22]:
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。")
df_file_info_after = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED_UPDATED.items()
])
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()

df_file_info = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED.items()
])
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).reset_index()
display(df_compare)
fig = px.line(df_compare, x="year", y="code_counts", color="file", line_dash="区分", markers=True)
fig.update_layout(title="処理前後のコード数推移", xaxis_title="年度", yaxis_title="コード数")
fig.show()


各ファイルの年推移をもう一度可視化し、改善していることを確かめます。


,file,year,code_counts,year_counts,column_counts,区分
0,fy-balance-sheet.csv,2010,3406,12,10,処理前
1,fy-balance-sheet.csv,2011,3513,12,10,処理前
2,fy-balance-sheet.csv,2012,3589,12,10,処理前
3,fy-balance-sheet.csv,2013,3679,12,10,処理前
4,fy-balance-sheet.csv,2014,3780,12,10,処理前
...,...,...,...,...,...,...
123,fy-stock-dividend.csv,2021,3618,12,8,処理後
124,fy-stock-dividend.csv,2022,3606,12,8,処理後
125,fy-stock-dividend.csv,2023,3606,12,8,処理後
126,fy-stock-dividend.csv,2024,3574,12,8,処理後


### 上場企業データのクリーニング

<summary><b>結果</b></summary>

</details>

In [23]:
print("\nロードします。")
df_code_info = dl.load_on_startup(settings["data_path"]["reference"], "", settings["files"]["reference"],header=0,na_values=settings["na_values"])

dl.chk_file_missing(df_code_info)

print("\n欠損値表現を確認します。")
df_missing_values_expression = eu.chk_missing_values_expression(
    df_code_info, filename=settings["files"]["reference"], option_value=""
)
#display(df_missing_values_expression)

print("\n読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ")
object_cols = df_code_info.select_dtypes(include='object').columns
df_code_info.loc[:, object_cols] = df_code_info.loc[:, object_cols].replace("-", np.nan)
#display(df_code_info)

print("\n数値列を Int64 に変換します。")
numeric_cols = ["17業種コード", "規模コード"]
df = df_code_info
df = cu.convert_columns_type(df, numeric_cols, "int", True)

print("\nstring 型に統一します。")
df = df.assign(**{col: df[col].astype("string") for col in df.columns if col not in numeric_cols})

print("\n日付列を datetime に変換します。")
df["日付"] = pd.to_datetime(df["日付"], errors="coerce", format="%Y%m%d")

df_type_change = df

print("\ndtypeの最終チェックをします。")
cu.chk_finale_dtype(df_type_change, settings["dtype_chk"]["reference"])

print("\n欠損値表現を確認します。")
df_missing_values_expression = eu.chk_missing_values_expression(df_type_change, filename=settings["files"]["reference"], option_value="")
#display(df_missing_values_expression)

print("\nデータ型を確認します。")
df_type = eu.chk_dtype(df=df_type_change,filename=settings["files"]["reference"],option_value="",na_drop=True)
#display(df_type)

#display(df_type_change)


ロードします。

欠損値表現を確認します。

読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ

数値列を Int64 に変換します。

列 '17業種コード' を int に変換中...

列 '規模コード' を int に変換中...

string 型に統一します。

日付列を datetime に変換します。

dtypeの最終チェックをします。

欠損値表現を確認します。

データ型を確認します。


In [24]:
from functools import reduce

df_list = [df for (filename, year) ,df in df_PROCESSED_UPDATED.items()]
dfs = [df.set_index(["コード", "年度"]) for df in df_list]  # df_list が64個のDF
df_ALL = reduce(lambda left, right: left.combine_first(right), dfs)
df_ALL.reset_index(inplace=True)

In [25]:
print("\nデータを最新上場企業のリストでフィルターします。")
df_filtered = df_ALL[~df_ALL["コード"].isin(df_type_change["コード"])]
display(df_filtered)


データを最新上場企業のリストでフィルターします。


,コード,年度,BPS,EPS,ROA,ROE,一株配当,利益剰余金,剰余金の配当,営業CF,...,純資産配当率,経常利益,総資産,総還元性向,自己資本比率,自社株買い,設備投資,財務CF,配当性向,長期借入金
55,1352,2010-03-01,628.95,27.09,1.9,4.31,20.0,1561000000.0,167000000.0,1370000000.0,...,<NA>,383000000.0,11967000000.0,<NA>,44.0,<NA>,-1985000000.0,-555000000.0,73.57,497000000.0
56,1352,2011-03-01,623.34,14.68,0.98,2.35,20.0,1517000000.0,167000000.0,726000000.0,...,<NA>,251000000.0,12534000000.0,<NA>,41.7,<NA>,-968000000.0,225000000.0,135.77,487000000.0
57,1352,2012-03-01,627.28,23.75,1.39,3.79,20.0,1549000000.0,167000000.0,1638000000.0,...,<NA>,505000000.0,14271000000.0,<NA>,36.8,<NA>,-472000000.0,-594000000.0,83.92,264000000.0
58,1352,2013-03-01,638.62,29.6,1.45,4.63,20.0,1630000000.0,167000000.0,60000000.0,...,3.1,516000000.0,17146000000.0,<NA>,31.2,<NA>,-3238000000.0,3087000000.0,67.3,3126000000.0
59,1352,2014-03-01,635.27,14.93,0.51,2.35,20.0,1590000000.0,167000000.0,656000000.0,...,3.1,290000000.0,24423000000.0,<NA>,21.8,<NA>,-5317000000.0,6415000000.0,133.1,4993000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62182,9995,2020-03-01,934.81,2.71,0.2,0.29,12.0,13864000000.0,312000000.0,1774000000.0,...,1.3,5000000.0,32061000000.0,437.5,70.3,<NA>,-80000000.0,-386000000.0,437.5,404000000.0
62183,9995,2021-03-01,924.83,-15.15,<NA>,<NA>,12.0,13184000000.0,312000000.0,-2591000000.0,...,1.3,-261000000.0,33653000000.0,<NA>,66.9,<NA>,-75000000.0,1260000000.0,-79.1,310000000.0
62184,9995,2022-03-01,902.95,-14.81,<NA>,<NA>,12.0,12506000000.0,312000000.0,-5163000000.0,...,1.3,1199000000.0,38682000000.0,<NA>,57.6,<NA>,-143000000.0,3761000000.0,-80.8,634000000.0
62185,9995,2023-03-01,872.09,34.18,2.31,3.53,12.0,13066000000.0,312000000.0,3858000000.0,...,1.4,1080000000.0,37784000000.0,35.1,65.3,<NA>,-160000000.0,-1970000000.0,35.1,335000000.0
